In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [38]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-torch',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 16 commands: 
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale2/0_0.0000" --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.0 --prompts="a circle;a square" --spatial_scales="0.015625;1.0" --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_spatial_softmax=0.0                   --rollout_steps=1024 --bptt_steps=16 --pool_size=1024 --bs=8 --lr=0.0003 --n_iters=200000 --clip_grad_norm=1.0
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale2/0_0.0010" --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.0 --prompts="a circle;a square" --spatial_scales="0.015625;1.0" --clip_model="clip-vit-base-patc

In [39]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24


In [40]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [41]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-torch
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/job_00003.status
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale2/0_0.0027" --dtype="float32" --device="cuda:0" --img_size=128 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.0 --prompts="a circle;a square" --spatial_scales="0.015625;1.0" --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_spatial_softmax=0.0026826957952797246 --rollout_steps=1024 --bptt_steps=16 --pool_size=1024 --bs=8 --lr=0.0003 --n_iters=200000 --clip_grad_norm=1.0 &> /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/job_00003.status

touch /home/akarshkumar0101/nca-alife-dat

In [42]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24/gpu_7.sh


# Track Experiment

In [59]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [60]:
(statuses==0).sum(), len(commands)

(16, 16)

In [61]:
np.unique(statuses, return_counts=True)

(array([0]), array([16]))

In [62]:
for job_id in list(range(len(commands))):
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        break
        print(e)


------------------------------------------------------ 0 ------------------------------------------------------

100%|██████████| 200000/200000 [3:41:22<00:00, 15.06it/s, loss=-0.289]

------------------------------------------------------ 1 ------------------------------------------------------

100%|██████████| 200000/200000 [3:39:46<00:00, 15.17it/s, loss=-0.297]

------------------------------------------------------ 2 ------------------------------------------------------

100%|██████████| 200000/200000 [3:45:02<00:00, 14.81it/s, loss=-0.288]

------------------------------------------------------ 3 ------------------------------------------------------

100%|██████████| 200000/200000 [3:47:04<00:00, 14.68it/s, loss=-0.287]

------------------------------------------------------ 4 ------------------------------------------------------

100%|██████████| 200000/200000 [3:42:35<00:00, 14.98it/s, loss=-0.294]

------------------------------------------------------ 5 -----------------

In [63]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_07_29_08_15_24'